In [2]:
#importing libaries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
#loading dataset
def remove_empty_and_space(text):
    return [lab.strip() for lab in text if lab]

df = pd.read_csv('/content/Evaluation-dataset.csv', header=None).fillna('')
df.columns = ['text'] + [f'label_{i}' for i in range(1, 15)]

df['target'] = df.iloc[:, 1:15].values.tolist()
df['target'] = df['target'].map(remove_empty_and_space)
df = df[['text', 'target']]
df['target_counts'] = df['target'].apply(len)

print(f"There are {df.shape[0]} Rows in Data")

There are 10132 Rows in Data


In [5]:
df.head()

,text,target,target_counts
0,Tires where delivered to the garage of my choi...,"[garage service positive, ease of booking posi...",2
1,"Easy Tyre Selection Process, Competitive Prici...","[garage service positive, value for money posi...",2
2,Very easy to use and good value for money.,[value for money positive],1
3,Really easy and convenient to arrange,[ease of booking positive],1
4,It was so easy to select tyre sizes and arrang...,"[location positive, value for money positive, ...",3


In [6]:
df.shape


(10132, 3)

In [7]:
import nltk
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Apply preprocessing to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Display the DataFrame
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,text,target,target_counts,cleaned_text
0,Tires where delivered to the garage of my choi...,"[garage service positive, ease of booking posi...",2,tires delivered garage choice garage notified ...
1,"Easy Tyre Selection Process, Competitive Prici...","[garage service positive, value for money posi...",2,easy tyre selection process competitive pricin...
2,Very easy to use and good value for money.,[value for money positive],1,easy use good value money
3,Really easy and convenient to arrange,[ease of booking positive],1,really easy convenient arrange
4,It was so easy to select tyre sizes and arrang...,"[location positive, value for money positive, ...",3,easy select tyre sizes arrange local fitting p...


In [8]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
tokens = tokenizer.batch_encode_plus(
    df['cleaned_text'].tolist(),
    max_length=128,
    padding='max_length',  # Updated parameter to avoid warnings
    truncation=True,
    return_tensors='pt'
)

# Extract token ids and attention masks
input_ids = tokens['input_ids']
attention_masks = tokens['attention_mask']

# Display the tokenized inputs
print(input_ids)
print(attention_masks)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([[  101, 13310,  5359,  ...,     0,     0,     0],
        [  101,  3733, 21904,  ...,     0,     0,     0],
        [  101,  3733,  2224,  ...,     0,     0,     0],
        ...,
        [  101,  3641, 21904,  ...,     0,     0,     0],
        [  101,  6581,  2326,  ...,     0,     0,     0],
        [  101, 25180,  3238,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
# Ensure that the 'target' column contains uniform data types (e.g., convert lists to strings or other uniform data types)
df['target'] = df['target'].apply(lambda x: str(x) if isinstance(x, list) else x)

# Encode the target labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['target'])

# Convert labels to tensor
labels_tensor = torch.tensor(labels, dtype=torch.long)

print(labels_tensor)


tensor([ 704,  756, 1353,  ...,  925, 1419, 1419])


In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode the target labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['target'])

# Convert labels to tensor
import torch
labels = torch.tensor(labels)


In [11]:
from torch.utils.data import DataLoader, TensorDataset, random_split

# Create a dataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [12]:
from transformers import BertForSequenceClassification

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

# Assuming `train_dataset` is already defined
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Increase step_size

# Set up mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Set up the training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def train_epoch(model, dataloader, optimizer, scheduler, scaler):
    model.train()
    total_loss = 0
    for batch in dataloader:
        b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss

        total_loss += loss.item()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Training loop
epochs = 1
for epoch in range(epochs):
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, scaler)
    print(f"Epoch {epoch + 1}, Loss: {train_loss:.2f}")

# Evaluate the model (assuming you have a validation function)
# validation_loss = evaluate(model, validation_dataloader)
# print(f"Validation Loss: {validation_loss:.2f}")


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1, Loss: 6.93


In [16]:
import torch
from sklearn.metrics import classification_report

def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    for batch in dataloader:
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
        logits = outputs.logits

        predictions.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(b_labels.tolist())
    return predictions, true_labels

# Evaluate the model
predictions, true_labels = evaluate(model, val_dataloader)

# Get unique classes from true_labels
unique_classes = sorted(set(true_labels))

# Check the number of unique classes
print(f"Number of unique classes in true labels: {len(unique_classes)}")
print(f"Number of target names: {len(label_encoder.classes_)}")

# Ensure target names match the unique classes
if len(unique_classes) != len(label_encoder.classes_):
    # Subset target names to match unique classes
    target_names = [label_encoder.classes_[i] for i in unique_classes]
else:
    target_names = label_encoder.classes_

# Print classification report


Number of unique classes in true labels: 449
Number of target names: 1420


In [17]:
# Example of interpreting results
# Print the first few predictions and actual labels
print("Predictions:", predictions[:5])
print("Actual Labels:", true_labels[:5])


Predictions: [1419, 1419, 1419, 1419, 1419]
Actual Labels: [452, 767, 1191, 1353, 1353]
